In [ ]:
pip install langchain python-dotenv langchain-groq tiktoken rapidocr-onnxruntime langchain-community langchain-text-splitters langchain-huggingface sentence-transformers

In [9]:
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant", temperature=0)
response=model.invoke("HI how are you")
print(response.content)

I'm just a computer program, so I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?


In [ ]:
pip install -qU langchain-chroma

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
pip install pypdf

In [52]:
from langchain_community.document_loaders import PyPDFLoader
import pprint

loader=PyPDFLoader("/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf")
docs=loader.load()
for doc in docs:
    pprint.pprint({
        "page": doc.metadata.get("page"),
        "content": doc.page_content[:500]  # first 500 chars
    })


{'content': 'Saathi: Government Scheme Information Handbook  \n'
            '1. Pradhan Mantri Awas Yojana (PMAY) \n'
            'About the Scheme: \n'
            'An initiative by the Government of India aimed at providing '
            'affordable housing for all by the \n'
            'year 2025. It targets urban and rural poor populations with '
            'financial assistance to build or \n'
            'purchase a home. \n'
            'Eligibility Criteria: \n'
            'Indian citizen aged 18 or above \n'
            'Annual household income below ₹18 lakhs \n'
            'Beneficiary should not own a pucca house in any part of India \n'
            'Priority given to women, SC',
 'page': 0}
{'content': 'Why is it beneficial? \n'
            'Provides financial protection from catastrophic health expenses '
            'and reduces out-of-pocket \n'
            'healthcare costs for the poor. \n'
            ' \n'
            '3. Pradhan Mantri Kisan Samman Nidhi (PM-K

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_docs=text_splitter.split_documents(docs)
split_docs

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-22T12:19:11+00:00', 'author': 'python-docx', 'moddate': '2025-06-22T12:19:11+00:00', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-22T12:19:11+00:00', 'author': 'python-docx', 'moddate': '2025-06-22T12:19:11+00:00', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='year 2025. It targets urban and rural poor populations with financial assistance to build or \npurchase a home. \nEligib

In [58]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="../langchain_db",  # Where to save data locally
)

In [59]:
vector_store.add_documents(split_docs)


['96282d8a-e3be-417c-8de2-1d75e81a81f7',
 '6eba6b2c-02aa-46a9-b1f3-5780c8db5c5a',
 '9abfef4f-3cf0-4e74-86d1-7b84a2f45334',
 'b57b9b4f-1de5-4762-806a-886a540e94ae',
 'a695d483-9df8-4d59-87c4-4048b0dba8a5',
 'f5177c6f-fce3-4560-9226-071f61e8d4cf',
 '14835b28-a460-468f-a7c9-fc7bd653dd01',
 '72954397-af42-49cb-9243-359349ed4995',
 '159c2280-9c02-44db-9876-41fc1737b0d8',
 'ae0966d3-270a-47bd-bf59-97f57966ac74',
 '7ad29527-fb44-4682-8921-5c92ea884c5a',
 '516bc4a1-a532-4dfb-9040-0fb96c8d0ca2',
 '5b040281-4752-4000-a154-3be9232a2e92',
 'ae35012a-f894-4726-8bb4-7782f1f6660d',
 '4636b9e2-2767-4738-9f5a-f2f2cfb9d217',
 'c627c775-a2a8-4415-8a1a-ebafb8906296',
 'd5b61e1b-6650-4458-911a-4707833699e0',
 '5e30d905-d8bd-431c-ba92-f2d62e7ebf54',
 '96fe6329-f164-4201-ac41-97120a96a07e']

In [60]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke("What is PMAY?")
docs

[Document(id='96282d8a-e3be-417c-8de2-1d75e81a81f7', metadata={'page': 0, 'moddate': '2025-06-22T12:19:11+00:00', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'page_label': '1', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-22T12:19:11+00:00', 'producer': 'www.ilovepdf.com', 'author': 'python-docx', 'total_pages': 3}, page_content='Saathi: Government Scheme Information Handbook  \n1. Pradhan Mantri Awas Yojana (PMAY) \nAbout the Scheme: \nAn initiative by the Government of India aimed at providing affordable housing for all by the'),
 Document(id='f5177c6f-fce3-4560-9226-071f61e8d4cf', metadata={'total_pages': 3, 'page_label': '1', 'producer': 'www.ilovepdf.com', 'author': 'python-docx', 'moddate': '2025-06-22T12:19:11+00:00', 'creationdate': '2025-06-22T12:19:11+00:00', 'creator': 'Microsoft® Word 2016', 'source': '/Users/adityaaryan/Documents/mlops/data/Saathi_Schemes_Info.pdf', 'page': 0}, page_content='2. Ayushman Bharat - Pradhan Mantr